In [1]:
import pandas as pd
import plotly.express as px
import altair as alt
from pyspark.ml.regression import LinearRegressionModel, DecisionTreeRegressionModel
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, monotonically_increasing_id, lit, date_add, explode, sequence
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import DoubleType
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType

In [2]:
sc = SparkContext.getOrCreate()

# Initialize SparkSession using the existing or new SparkContext
spark = SparkSession.builder \
        .appName("Demo App") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()


24/12/18 17:20:44 WARN Utils: Your hostname, Khim3 resolves to a loopback address: 127.0.1.1; using 10.0.111.51 instead (on interface wlo1)
24/12/18 17:20:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/18 17:20:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/18 17:20:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
path = 'models/linear_regressor'
model = LinearRegressionModel.load(path)

In [4]:
data_test = spark.read.csv('./test.csv', header=True, inferSchema=True)

data_test.show(5)

+----------+----------+----------+----------+----------+----------+--------+
|      Date|      Open|      High|       Low|     Close| Adj Close|  Volume|
+----------+----------+----------+----------+----------+----------+--------+
|2018-02-07|266.579987|272.450012|264.329987|264.559998|264.559998| 8981500|
|2018-02-08|267.079987|267.619995|     250.0|250.100006|250.100006| 9306700|
|2018-02-20| 277.73999|285.809998|276.609985|278.549988|278.549988| 7769000|
|2018-02-26|    288.75|295.649994| 287.01001|294.160004|294.160004|10268600|
|2018-03-05|302.850006|316.910004|297.600006|     315.0|     315.0|18986100|
+----------+----------+----------+----------+----------+----------+--------+
only showing top 5 rows



In [5]:
# check data type of data_tetst
data_test.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: integer (nullable = true)



24/12/18 17:20:56 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [6]:
if "features" in data_test.columns:
    data_test = data_test.drop("features")
feature_columns = ["Open", "High", "Low", "Volume"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df = assembler.transform(data_test).select("features", col("Close").alias("label"))
df.show(5)

+--------------------+----------+
|            features|     label|
+--------------------+----------+
|[266.579987,272.4...|264.559998|
|[267.079987,267.6...|250.100006|
|[277.73999,285.80...|278.549988|
|[288.75,295.64999...|294.160004|
|[302.850006,316.9...|     315.0|
+--------------------+----------+
only showing top 5 rows



In [7]:
df = df.orderBy(col("features")).limit(1)
predictions = model.transform(df)        
predictions.show()


+--------------------+------+------------------+
|            features| label|        prediction|
+--------------------+------+------------------+
|[255.710007,266.6...|264.75|262.95816627665533|
+--------------------+------+------------------+



24/12/18 17:21:31 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
